# Earthquake data

#### Load Python tools

In [2]:
%load_ext lab_black

In [3]:
import pandas as pd
import geopandas as gpd

In [4]:
pd.options.display.max_columns = 1000
pd.options.display.max_rows = 1000

---

## Read data

#### Northridge quake [impact area data](https://services.arcgis.com/QJfoC7c7Z2icolha/arcgis/rest/services/impact_area1375916377789/FeatureServer/0)  from Esri

In [ ]:
northridge_src.plot()

In [ ]:
northridge_src.columns

In [ ]:
# Esri's metadata here: https://services.arcgis.com/QJfoC7c7Z2icolha/arcgis/rest/services/impact_area1375916377789/FeatureServer/0

#### Just get the number of people, businesses and infrastructure

In [ ]:
northridge_src_slim = northridge_src[
    ["GRID_CODE", "CIKR_Total", "BUS_Total", "TOTPOP_CY_2", "geometry"]
].copy()

#### Rename columns

In [ ]:
northridge_src_slim.rename(
    columns={
        "GRID_CODE": "impact_code",
        "CIKR_Total": "critical_infrastructure",
        "BUS_Total": "businesses",
        "TOTPOP_CY_2": "population",
    },
    inplace=True,
)

#### Add descriptions to impact codes

In [ ]:
northridge_src_slim["impact_code"] = northridge_src_slim["impact_code"].astype(str)

In [ ]:
codes = {
    "4": "Shaking, no damage",
    "5": "Very light",
    "6": "Light",
    "7": "Moderate",
    "8": "Moderate/Heavy",
    "9": "Heavy",
}

In [ ]:
northridge_src_slim["impact_code_desc"] = northridge_src_slim["impact_code"].map(codes)

#### Prepare for export

In [ ]:
northridge_src_slim = northridge_src_slim[
    [
        "impact_code",
        "impact_code_desc",
        "critical_infrastructure",
        "businesses",
        "population",
        "geometry",
    ]
]

#### Heaviest area?

In [ ]:
northridge_src_slim[northridge_src_slim["impact_code"] == "9"].plot()

#### How many people lived there? 

In [ ]:
northridge_src_slim[northridge_src_slim["impact_code"] == "9"]["population"].iloc[0]

---

## Historical earthquakes

#### L.A. area quakes >= 5 magnitude

In [ ]:
with open("data/raw/query_mag_3.json") as f:
    gj = geojson.load(f)

In [ ]:
features = gj["features"]

In [ ]:
la_quakes_gdf = gpd.GeoDataFrame(features)

#### Unpack nested properties

In [ ]:
la_quakes_gdf[
    [
        "mag",
        "place",
        "time",
        "updated",
        "tz",
        "url",
        "detail",
        "felt",
        "cdi",
        "mmi",
        "alert",
        "status",
        "tsunami",
        "sig",
        "net",
        "code",
        "ids",
        "sources",
        "types",
        "nst",
        "dmin",
        "rms",
        "gap",
        "magType",
        "type",
        "title",
    ]
] = pd.json_normalize(la_quakes_gdf["properties"])

In [ ]:
la_quakes_gdf["year"] = pd.to_datetime(la_quakes_gdf["time"], unit="ms").dt.strftime(
    "%Y"
)
la_quakes_gdf["date"] = pd.to_datetime(la_quakes_gdf["time"], unit="ms").dt.strftime(
    "%Y-%m-%d"
)

In [ ]:
la_quakes_gdf.drop(
    [
        "properties",
        "type",
        "updated",
        "tz",
        "url",
        "detail",
        "status",
        "net",
        "code",
        "ids",
        "sources",
        "types",
        "alert",
    ],
    axis=1,
    inplace=True,
)

In [ ]:
la_quakes_gdf.plot()

In [ ]:
la_quakes_gdf["latitude"] = la_quakes_gdf["geometry"].x
la_quakes_gdf["longitude"] = la_quakes_gdf["geometry"].y

In [ ]:
la_quakes_gdf = la_quakes_gdf[
    [
        "id",
        "mag",
        "place",
        "year",
        "date",
        "latitude",
        "longitude",
        "title",
        "geometry",
    ]
]

In [ ]:
la_quakes_gdf.head()

---

## Export

In [ ]:
northridge_src_slim.to_file(
    "../../usc/data/geo/northridge_quake_boundaries.geojson", driver="GeoJSON"
)

In [ ]:
la_quakes_gdf.to_file(
    "../../usc/data/geo/la-area-quakes-1950-2022.geojson", driver="GeoJSON"
)